In [28]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [29]:
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 64. Light and variable wind becoming northwest 5 to 10 mph in the morning. Winds could gust as high as 20 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 64. Light and variable wind becoming northwest 5 to 10 mph in the morning. Winds could gust as high as 20 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 64 °F
 </p>
</div>


In [30]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

Today
Sunny
High: 64 °F


In [31]:
img = tonight.find("img")
desc = img['title']
print(desc)

Today: Sunny, with a high near 64. Light and variable wind becoming northwest 5 to 10 mph in the morning. Winds could gust as high as 20 mph. 


In [32]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight',
 'Monday',
 'MondayNight',
 'Tuesday']

In [33]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Sunny', 'Clear', 'Sunny', 'Mostly Clear', 'Sunny', 'Mostly Clear', 'Mostly Sunny', 'Mostly Clear', 'Mostly Sunny']
['High: 64 °F', 'Low: 46 °F', 'High: 63 °F', 'Low: 44 °F', 'High: 64 °F', 'Low: 45 °F', 'High: 62 °F', 'Low: 45 °F', 'High: 61 °F']
['Today: Sunny, with a high near 64. Light and variable wind becoming northwest 5 to 10 mph in the morning. Winds could gust as high as 20 mph. ', 'Tonight: Clear, with a low around 46. East northeast wind 8 to 14 mph, with gusts as high as 26 mph. ', 'Saturday: Sunny, with a high near 63. Northeast wind 6 to 9 mph. ', 'Saturday Night: Mostly clear, with a low around 44. Northeast wind around 7 mph. ', 'Sunday: Sunny, with a high near 64. North northeast wind around 8 mph. ', 'Sunday Night: Mostly clear, with a low around 45.', 'Monday: Mostly sunny, with a high near 62.', 'Monday Night: Mostly clear, with a low around 45.', 'Tuesday: Mostly sunny, with a high near 61.']


In [34]:
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 64 °F,"Today: Sunny, with a high near 64. Light and v..."
1,Tonight,Clear,Low: 46 °F,"Tonight: Clear, with a low around 46. East nor..."
2,Saturday,Sunny,High: 63 °F,"Saturday: Sunny, with a high near 63. Northeas..."
3,SaturdayNight,Mostly Clear,Low: 44 °F,"Saturday Night: Mostly clear, with a low aroun..."
4,Sunday,Sunny,High: 64 °F,"Sunday: Sunny, with a high near 64. North nort..."
5,SundayNight,Mostly Clear,Low: 45 °F,"Sunday Night: Mostly clear, with a low around 45."
6,Monday,Mostly Sunny,High: 62 °F,"Monday: Mostly sunny, with a high near 62."
7,MondayNight,Mostly Clear,Low: 45 °F,"Monday Night: Mostly clear, with a low around 45."
8,Tuesday,Mostly Sunny,High: 61 °F,"Tuesday: Mostly sunny, with a high near 61."


In [36]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    64
1    46
2    63
3    44
4    64
5    45
6    62
7    45
8    61
Name: temp_num, dtype: object

In [37]:
weather["temp_num"].mean()


54.888888888888886

In [38]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [39]:
weather[is_night]


,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Clear,Low: 46 °F,"Tonight: Clear, with a low around 46. East nor...",46,True
3,SaturdayNight,Mostly Clear,Low: 44 °F,"Saturday Night: Mostly clear, with a low aroun...",44,True
5,SundayNight,Mostly Clear,Low: 45 °F,"Sunday Night: Mostly clear, with a low around 45.",45,True
7,MondayNight,Mostly Clear,Low: 45 °F,"Monday Night: Mostly clear, with a low around 45.",45,True
